# SARIMA Models

## Autoregressive (AR) Models
An AR model of order $p$, denoted AR($p$), is a model where the current time point is dependent on the previous $p$ time points. Formally, the model is expressed as
$$
X_t = c + \sum_{i=1}^p \alpha_i X_{t-i} + \epsilon_t
$$
where $X_{t-i}$ are lag variables, $c$ is a constant, $\alpha_i$ are paremeters, and $\epsilon_t$ is white noise.

Equivalently, the model can be expressed with lag operators which maps current time points to previous time points. Lag operators $L^i$ are defined by
$$
L^i[X_t] = X_{t-i}
$$
which can be used to give us the expression
$$
\left(1 - \sum_{i=1}^p \alpha_i L^i \right) X_t = c + \epsilon_t
$$

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

<IPython.core.display.Javascript object>

## Moving-Average (MA) Models
An MA model of order $q$, denoted MA($q$), is a model where the current time point is dependent on the previous $q$ noise terms. Formally, the model is expressed as
$$
X_t = \mu + \epsilon_t + \sum_{i=1}^q \beta_i \epsilon_{t-i}
$$
where $\mu = \mathrm{E}[X_t]$, $\beta_i$ are additional model parameters, and $\epsilon_{t-i}$ are again white noise. Note, it is assumed $\epsilon_t \overset{iid}{\sim} \mathcal{N}(0, \sigma^2)$ for all $t$.

## Autoregressive Moving-Average (ARMA) Models
An ARMA model of order ($p$, $q$), denoted ARMA($p$, $q$), is a model where the current time point is dependent on the previous $p$ time points and previous $q$ noise terms. Formally, the model is expressed as
$$
X_t = c + \epsilon_t + \sum_{i=1}^p \alpha_i X_{t-i} + \sum_{i=1}^q \beta_i \epsilon_{t-i}
$$
or equivalently
$$
\left(1 - \sum_{i=1}^p \alpha_i L^i \right)X_t = \left(1 + \sum_{i=1}^q \beta_i L^i \right) \epsilon_t
$$

## Autoregressive Integrated Moving-Average (ARIMA) Models
An ARIMA model of order ($p$, $d$, $q$), denoted ARIMA($p$, $d$, $q$), is like an ARMA($p+d$, $q$) model where it is assumed that the polynomial $\left(1 - \sum_{i=1}^p \alpha_i L^i\right)$ has a unit root, i.e. a factor of $(1 - L)$ of multiplicity $d$.

An ARMA($p'$, $q$) model is written
$$
\left(1 - \sum_{i=1}^{p'} \alpha_i L^i \right)X_t = \left(1 + \sum_{i=1}^q \beta_i L^i \right) \epsilon_t
$$
but with the polynomial factorization property, $(1-L)$ can be factored out $d$ times from the the polynomial on the LHS, giving
$$
\left(1 - \sum_{i=1}^{p'} \alpha_i L^i \right) = \left(1 - \sum_{i=1}^{p'-d} \gamma_i L^i \right)(1-L)^d
$$
The ARIMA($p$, $d$, $q$) model with $p = p' - d$ is thus written
$$
\left(1 - \sum_{i=1}^{p} \gamma_i L^i \right)(1-L)^d X_t = \left(1 + \sum_{i=1}^q \beta_i L^i \right) \epsilon_t
$$
When there is a constant $c$ introduced into the model, then the ARIMA($p$, $d$, $q$) model is said to have drift:
$$
\left(1 - \sum_{i=1}^{p} \gamma_i L^i \right)(1-L)^d X_t = c + \left(1 + \sum_{i=1}^q \beta_i L^i \right) \epsilon_t
$$
giving a drift of $\frac{c}{1 - \sum_{i=1}^p \gamma_i}$.